### Data Preprocessing
You can use your own way of preprocessing to enhance results. Best results will lead to bonus points.

In [1]:
import pandas as pd
import nltk
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np

In [2]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:00<00:00, 114MB/s] 
100% 25.7M/25.7M [00:00<00:00, 101MB/s]


In [3]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [2]:
reviews = pd.read_csv("/content/IMDB Dataset.csv")

In [3]:
reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
reviews.sentiment = reviews.sentiment.map({'positive':1,'negative':0})

In [5]:
reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [6]:
reviews.isna().sum()

review       0
sentiment    0
dtype: int64

In [7]:
reviews.duplicated().sum()

418

In [8]:
reviews.drop_duplicates(inplace=True)

In [9]:
reviews.reset_index(drop=True, inplace=True)

In [10]:
reviews.duplicated().sum()

0

In [11]:
reviews.shape

(49582, 2)

In [12]:
reviews["length"] = reviews.review.str.len()

In [13]:
reviews.head()

,review,sentiment,length
0,One of the other reviewers has mentioned that ...,1,1761
1,A wonderful little production. <br /><br />The...,1,998
2,I thought this was a wonderful way to spend ti...,1,926
3,Basically there's a family where a little boy ...,0,748
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,1317


In [14]:
reviews.describe()

,sentiment,length
count,49582.000000,49582.000000
mean,0.501876,1310.568230
std,0.500002,990.762238
min,0.000000,32.000000
25%,0.000000,699.000000
50%,1.000000,971.000000
75%,1.000000,1592.000000
max,1.000000,13704.000000


In [15]:
reviews_list = reviews.review.to_list()

In [16]:
len(reviews_list[0].split())

307

In [17]:
reviews.to_csv("clean.csv", index=False)

In [18]:
for i in range(len(reviews_list)):
    words = reviews_list[i].split()
    if len(words) > 200:
        reviews_list[i] = ' '.join(words[:200])

Crearing Tokenizer

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [20]:
tokenizer = Tokenizer()

In [21]:
len(reviews_list)

49582

In [22]:
tokenizer.fit_on_texts(reviews_list)

In [23]:
len(tokenizer.word_index)

103342

In [24]:
X = tokenizer.texts_to_sequences(reviews_list)

In [25]:
Y = reviews.sentiment.to_list()

In [26]:
len(X), len(Y)

(49582, 49582)

In [27]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, Y, test_size=0.5, random_state=1)

In [28]:
len(val_X), len(val_y), len(train_X), len(train_y)

(24791, 24791, 24791, 24791)

In [29]:
test_X, val_X, test_y, val_y = train_test_split(val_X, val_y, test_size=0.2, random_state=1)

In [30]:
len(val_X), len(val_y), len(test_X), len(test_y)

(4959, 4959, 19832, 19832)

In [31]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [32]:
padded_reviews = pad_sequences(train_X, maxlen=200, padding='pre')

In [33]:
len(padded_reviews)

24791

In [34]:
train_X = padded_reviews

In [61]:
val_X = pad_sequences(val_X, maxlen=200, padding='pre')

In [68]:
for i in train_X:
    if len(i) > 200:
        print(len(i))

DataSet Class

In [36]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x_data, y_data, transform=None):
        self.reviews = x_data
        self.labels = y_data
        self.transform = transform

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
      try:
        sample = {'review': self.reviews[idx], 'label': self.labels[idx]}
      except:
        sample = {'review': np.zeros_like(self.reviews[0]), 'label': 0}

      if self.transform:
          sample = self.transform(sample)

      return sample

### Sequence models
Following is an example code for simple LSTMs containing one layer. Your implementation should be generic in which user can be able to create multiple layers if required.

In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm

class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, vocab_size):
        super(LSTM, self).__init__()
        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, input_dim)

        self.W_x = nn.Linear(input_dim, 4 * hidden_dim, bias=True)
        self.W_h = nn.Linear(hidden_dim, 4 * hidden_dim, bias=True)

        self.drop_layer = nn.Dropout(p=0.25)
        self.output_layer = nn.Linear(hidden_dim, 1)  # Output layer for binary classification

        self.Sigmoid = nn.Sigmoid()
        self.Tanh = nn.Tanh()
        self.h = None
        self.c = None

    def lstm_step(self, inp, prev_hidden_cell):
        h_prev, c_prev = prev_hidden_cell

        inp = self.embedding(inp)

        inp = self.drop_layer(inp)

        activation = self.W_x(inp) + self.W_h(h_prev)

        ai, af, ac, ao = activation.chunk(4, 1)

        in_gate = self.Sigmoid(ai)
        forget_gate = self.Sigmoid(af)
        cell_gate = self.Tanh(ac)
        out_gate = self.Sigmoid(ao)
        updated_c = forget_gate * c_prev + in_gate * cell_gate
        updated_h = out_gate * self.Tanh(updated_c)
        return updated_h, updated_c

    def forward(self, inp):
        batch_size, seq_len = inp.shape
        device = inp.device

        if self.h is None or self.c is None:
            h = torch.zeros(batch_size, self.hidden_dim).to(device)
            c = torch.zeros(batch_size, self.hidden_dim).to(device)
        else:
            h = self.h.detach().to(device)
            c = self.c.detach().to(device)

        for t in range(seq_len):
            x_t = inp[:, t]
            h, c = self.lstm_step(x_t, (h, c))

        self.h, self.c = h, c  # Update the internal states

        out = self.output_layer(h)
        return out

# Train

In [39]:
from tqdm import tqdm

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
device

device(type='cuda')

In [42]:
from google.colab import files

In [59]:
import torch
import numpy as np
from tqdm import tqdm

def train(model, num_epochs, criterion, optimizer, train_loader, val_loader, device):
    history = {'train_loss': [], 'val_loss': [], 'train_accuracy': [], 'val_accuracy': [], 'best_accuracy': -np.inf, 'min_loss': np.inf}

    for epoch in range(num_epochs):
        model.train()
        train_loss, train_corrects = 0, 0

        outer = tqdm(total=len(train_loader.dataset), desc=f'Train Epoch: {epoch + 1} / {num_epochs}')
        for batch in train_loader:
            try:
                inputs = batch['review'].to(device).long()
                labels = batch['label'].view(-1, 1).to(device).float()  # Convert labels to float

                optimizer.zero_grad()
                output = model(inputs)

                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

                # Use sigmoid to convert outputs to probabilities
                output_probs = torch.sigmoid(output)
                predicted = (output_probs > 0.5).float()  # Binary classification
                train_corrects += (predicted == labels).sum().item()

                outer.update(len(inputs))
            except Exception as e:
                print(f"Error: {e}")
                outer.update(len(inputs))

        outer.close()

        train_accuracy = 100. * train_corrects / len(train_loader.dataset)
        train_loss /= len(train_loader)
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Train Accuracy: {train_accuracy:.2f}%")

        history['train_loss'].append(train_loss)
        history['train_accuracy'].append(train_accuracy)

        # Save the model state
        torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')

        # Validation
        model.eval()
        val_loss, val_corrects = 0, 0

        with torch.no_grad():
            outer = tqdm(total=len(val_loader.dataset), desc='Validating')
            for batch in val_loader:
                inputs = batch['review'].to(device).long()
                labels = batch['label'].view(-1, 1).to(device).float()  # Convert labels to float

                output = model(inputs)
                loss = criterion(output, labels)

                val_loss += loss.item()

                # Use sigmoid to convert outputs to probabilities
                output_probs = torch.sigmoid(output)
                predicted = (output_probs > 0.5).float()  # Binary classification
                val_corrects += (predicted == labels).sum().item()

                outer.update(len(inputs))

            outer.close()

            val_accuracy = 100. * val_corrects / len(val_loader.dataset)
            val_loss /= len(val_loader)
            print(f"Val Loss: {val_loss:.4f}")
            print(f"Val Accuracy: {val_accuracy:.2f}%")

            history['val_loss'].append(val_loss)
            history['val_accuracy'].append(val_accuracy)

            if val_accuracy > history['best_accuracy']:
                history['best_accuracy'] = val_accuracy

            if val_loss < history['min_loss']:
                history['min_loss'] = val_loss

    return history

In [70]:
trainLoader = CustomDataset(train_X, train_y)
valLoader = CustomDataset(val_X, val_y)
# testLoader = CustomDataset(test_x, test_y, transform)

In [71]:
train_loader = torch.utils.data.DataLoader(trainLoader, batch_size=256, shuffle=True, drop_last = True)
val_loader = torch.utils.data.DataLoader(valLoader, batch_size=256, shuffle=True, drop_last= True)
# test_loader = torch.utils.data.DataLoader(testLoader, batch_size=32, shuffle=True)

In [72]:
criterion = nn.BCEWithLogitsLoss()
model = LSTM(200,256, 103343)

In [73]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))


True
0
Tesla T4


In [74]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [75]:
model.to(device)

LSTM(
  (embedding): Embedding(103343, 200)
  (W_x): Linear(in_features=200, out_features=1024, bias=True)
  (W_h): Linear(in_features=256, out_features=1024, bias=True)
  (drop_layer): Dropout(p=0.25, inplace=False)
  (output_layer): Linear(in_features=256, out_features=1, bias=True)
  (Sigmoid): Sigmoid()
  (Tanh): Tanh()
)

In [76]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
history = train(model, 10, criterion, optimizer, train_loader, val_loader, device)




Train Epoch: 1 / 50:   0%|          | 0/24791 [00:00<?, ?it/s]


Train Epoch: 1 / 50:   1%|          | 256/24791 [00:00<00:42, 581.21it/s]


Train Epoch: 1 / 50:   2%|▏         | 512/24791 [00:00<00:40, 593.04it/s]


Train Epoch: 1 / 50:   3%|▎         | 768/24791 [00:01<00:39, 602.60it/s]


Train Epoch: 1 / 50:   4%|▍         | 1024/24791 [00:01<00:39, 604.95it/s]


Train Epoch: 1 / 50:   5%|▌         | 1280/24791 [00:02<00:38, 606.47it/s]


Train Epoch: 1 / 50:   6%|▌         | 1536/24791 [00:02<00:38, 609.22it/s]


Train Epoch: 1 / 50:   7%|▋         | 1792/24791 [00:02<00:38, 602.67it/s]


Train Epoch: 1 / 50:   8%|▊         | 2048/24791 [00:03<00:42, 530.96it/s]


Train Epoch: 1 / 50:   9%|▉         | 2304/24791 [00:04<00:41, 546.92it/s]


Train Epoch: 1 / 50:  10%|█         | 2560/24791 [00:04<00:42, 527.68it/s]


Train Epoch: 1 / 50:  11%|█▏        | 2816/24791 [00:05<00:43, 507.33it/s]


Train Epoch: 1 / 50:  12%|█▏        | 3072/24791 [00:05<00:41, 528.88it/s]


Train Epoch

Train Loss: 0.6465
Train Accuracy: 60.80%





Validating:   0%|          | 0/4959 [00:00<?, ?it/s]


Validating:  10%|█         | 512/4959 [00:00<00:01, 3731.43it/s]


Validating:  21%|██        | 1024/4959 [00:00<00:01, 3831.16it/s]


Validating:  31%|███       | 1536/4959 [00:00<00:00, 3699.15it/s]


Validating:  41%|████▏     | 2048/4959 [00:00<00:00, 3875.86it/s]


Validating:  52%|█████▏    | 2560/4959 [00:00<00:00, 3937.45it/s]


Validating:  62%|██████▏   | 3072/4959 [00:00<00:00, 3910.75it/s]


Validating:  72%|███████▏  | 3584/4959 [00:00<00:00, 3928.69it/s]


Validating:  83%|████████▎ | 4096/4959 [00:01<00:00, 3745.50it/s]


Validating:  98%|█████████▊| 4864/4959 [00:01<00:00, 3827.50it/s]


Val Loss: 0.6841
Val Accuracy: 62.49%





Train Epoch: 2 / 50:   0%|          | 0/24791 [00:00<?, ?it/s]


Train Epoch: 2 / 50:   1%|          | 256/24791 [00:00<00:41, 592.42it/s]


Train Epoch: 2 / 50:   2%|▏         | 512/24791 [00:00<00:40, 602.75it/s]


Train Epoch: 2 / 50:   3%|▎         | 768/24791 [00:01<00:39, 606.90it/s]


Train Epoch: 2 / 50:   4%|▍         | 1024/24791 [00:01<00:38, 611.27it/s]


Train Epoch: 2 / 50:   5%|▌         | 1280/24791 [00:02<00:38, 613.02it/s]


Train Epoch: 2 / 50:   6%|▌         | 1536/24791 [00:02<00:37, 612.27it/s]


Train Epoch: 2 / 50:   7%|▋         | 1792/24791 [00:02<00:37, 612.92it/s]


Train Epoch: 2 / 50:   8%|▊         | 2048/24791 [00:03<00:37, 600.69it/s]


Train Epoch: 2 / 50:   9%|▉         | 2304/24791 [00:03<00:38, 590.55it/s]


Train Epoch: 2 / 50:  10%|█         | 2560/24791 [00:04<00:38, 582.07it/s]


Train Epoch: 2 / 50:  11%|█▏        | 2816/24791 [00:04<00:38, 567.89it/s]


Train Epoch: 2 / 50:  12%|█▏        | 3072/24791 [00:05<00:38, 562.56it/s]


Train Epoch

...

Train Loss: 0.0305
Train Accuracy: 98.15%


Validating:  98%|█████████▊| 4864/4959 [00:01<00:00, 4105.88it/s]


Val Loss: 0.8041
Val Accuracy: 82.74%


Train Epoch: 23 / 50:  19%|█▊        | 4608/24791 [00:07<00:33, 595.19it/s]

KeyboardInterrupt: 

In [ ]:
with open ("results.json", "w") as f:
    import json
    json.dump(history, f)

# Testing

In [77]:
model = LSTM(200,256, 103343)
model.load_state_dict(torch.load('model_epoch_11.pth'))
model.to(device)

LSTM(
  (embedding): Embedding(103343, 200)
  (W_x): Linear(in_features=200, out_features=1024, bias=True)
  (W_h): Linear(in_features=256, out_features=1024, bias=True)
  (drop_layer): Dropout(p=0.25, inplace=False)
  (output_layer): Linear(in_features=256, out_features=1, bias=True)
  (Sigmoid): Sigmoid()
  (Tanh): Tanh()
)

In [81]:
inp = input("Enter review: ")
inp = tokenizer.texts_to_sequences([inp])
inp = pad_sequences(inp, maxlen=200, padding='pre')
inp = torch.tensor(inp).to(device)
output = model(inp)
output_probs = torch.sigmoid(output)
predicted = (output_probs > 0.5).float()
if predicted == 1:
    print("Positive")
else:
    print("Negative")

Enter review: driving a bike
Negative
